In [2]:
%load_ext autoreload
%autoreload 2



In [3]:
from runner import *
from process_results import *
import os
import pandas as pd

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:

data = {
    "name": [],
    "num_exce": [],
    "num_frame": [],
    "is_async": [],
    "logged": [],
    "dynamic": [],
    "static": [],
    "hybrid_online": [],
    "hybrid_offline": [],
    "distance": [], 
    "running_service": []
}

for name, app in TEST_APPLICATIONS.items():
    data["name"].append(name)
    ground_truth = app.read_ground_truth()
    dynamic_dependency = app.read_latest_dynamic_dependency()
    root_cause_in_log = check_root_cause_in_log(ground_truth, app.origin_log_path)
    data["is_async"].append(app.is_async)
    data["logged"].append(root_cause_in_log)

    (tp, fp, fn) = process_dependency_result(dynamic_dependency, ground_truth)
    data["dynamic"].append(f"{tp}/{fp}/{fn}")

    static_dependency = app.read_latest_static_dependency()
    (tp, fp, fn) = process_dependency_result(static_dependency, ground_truth)
    data["static"].append(f"{tp}/{fp}/{fn}")

    hybrid_online, hybrid_offline = app.read_latest_hybrid_dependency()
    (tp, fp, fn) = process_dependency_result(hybrid_online, ground_truth)
    data["hybrid_online"].append(f"{tp}/{fp}/{fn}")
    (tp, fp, fn) = process_dependency_result(hybrid_offline, ground_truth)
    data["hybrid_offline"].append(f"{tp}/{fp}/{fn}")
    data["distance"].append(get_exception_distance(ground_truth,
                                                   os.path.join(app.get_latest_result("dynamic"), "exception.json")))


    dynamic_result_path = app.get_latest_result("dynamic")
    exceptions = read_exceptions(os.path.join(dynamic_result_path, "exception.json"))
    data['num_exce'].append(len(exceptions))
    data["num_frame"].append(len(open(os.path.join(app.get_latest_result(
        "dynamic"), "affected-var-results.json")).readlines()))

    data['running_service'].append(app.is_running_service)

df = pd.DataFrame(data)
print(df)

             name  num_exce  num_frame  is_async  logged dynamic static  \
0        jena_324      1383       8310      True    True   1/0/0  0/1/1   
1    tomcat_65131        23         84     False   False   1/0/0  0/0/1   
2   fineract_1211        77        311     False    True   1/0/0  1/0/0   
3  mapreduce_6654       124        486     False    True   2/0/0  0/0/2   
4       nifi_8249        79       2642      True    True   2/0/0  0/3/2   
5      solr_16363       442       1927      True   False   1/0/0  0/1/1   
6    hadoop_17812        33        127     False   False   1/0/0  0/1/1   
7     wicket_6249        13        103      True   False   1/0/0  1/0/0   
8       hdfs_4128       124        473      True   False   1/0/0  0/0/1   
9      hive_13410        49        278      True   False   5/0/0  0/0/5   

  hybrid_online hybrid_offline  distance  running_service  
0         1/0/0          0/1/1      1378             True  
1         0/0/1          0/0/1         0             T

In [1]:

data = {
    "name": [],
    "type": [],
    "origin": [],
    "static": [],
    "hybrid": [], 
    "dynamic": []
}
for name, app in BENCH_APPLICATIONS.items():
    separate_results = ["fineract_bench", "jena_bench"]
    print(name)

    origin_perf_result = app.perf_result_path("origin")
    static_perf_result = app.perf_result_path("static")
    hybrid_perf_result = app.perf_result_path("hybrid")
    dynamic_perf_result = app.perf_result_path("dynamic")

    if name in separate_results:
        result = read_separate_perf_result([origin_perf_result, static_perf_result, hybrid_perf_result, dynamic_perf_result])
    else:
        result = read_aggregate_perf_result([origin_perf_result, static_perf_result, hybrid_perf_result, dynamic_perf_result])
    
    for key, value in result.items():
        if key == "tpr":
            m = "ms"
        if key == "throughput":
            m = "op/s"
            key = "tp"
        if key == "exec_time":
            m = "s"
            key = "time"
        data["name"].append(name)
        data["type"].append(key + f"({m})")
        data["origin"].append(value[0])
        data["static"].append(value[1])
        if name == "solr_bench":
            data["hybrid"].append("XXXX")
        else:
            data["hybrid"].append(value[2])
        data["dynamic"].append(value[3])
df = pd.DataFrame(data)
print(df)
    



NameError: name 'BENCH_APPLICATIONS' is not defined